In [ ]:
from selenium import webdriver
from fake_useragent import UserAgent
import time

In [ ]:

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
user_ag = UserAgent().random
options.add_argument('user-agent=%s'%user_ag)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("prefs", {"prfile.managed_default_content_setting.images": 2})
driver = webdriver.Chrome('chromedriver.exe', options=options)

# 크롤링 방지 설정을 undefined로 변경
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            })
            """
})

url = 'https://login.coupang.com/login/login.pang'
driver.get(url=url)
time.sleep(2)

id_input = driver.find_element_by_xpath('//*[@id="login-email-input"]')
id_input.send_keys('쿠팡아이디')

pw_input = driver.find_element_by_xpath('//*[@id="login-password-input"]')
pw_input.send_keys('쿠팡패스워드')

driver.find_element_by_xpath('/html/body/div[1]/div/div/form/div[5]/button').click()

In [16]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import ssl
import urllib3
import re
urllib3.disable_warnings()

In [18]:
# &page=1

# &listSize=72

In [19]:
class_names_eng = ['blouse', 'cardigan', 'coat', 'hoodie', 'jacket', 'jumper','jumpsuit',
                   'knit', 'longpants', 'onepiece', 'shirt', 'shortpants', 'skirt', 'tshirt']

In [20]:
class_names_kor = ['블라우스', '가디건', '코트', '후드티', '자켓', '점퍼', '점프수트',
                   '니트', '긴바지', '원피스', '셔츠', '반바지', '치마', '티셔츠']

In [21]:
filter_color = {
    "블랙" : "&filter=1%23attr_10496%244278%40DEFAULT",
    "화이트" : "&filter=1%23attr_10496%244289%40DEFAULT",
    "레드" : "&filter=1%23attr_10496%244282%40DEFAULT",
    "블루" : "&filter=1%23attr_10496%244286%40DEFAULT",
    "옐로우" : "&filter=1%23attr_10496%244284%40DEFAULT",
    "그린" : "&filter=1%23attr_10496%244285%40DEFAULT",
    "퍼플" : "&filter=1%23attr_10496%244287%40DEFAULT",
    "브라운" : "&filter=1%23attr_10496%244290%40DEFAULT",
    "네이비" : "&filter=1%23attr_10496%244279%40DEFAULT",
    "오렌지" : "&filter=1%23attr_10496%244283%40DEFAULT",
    "그레이" : "&filter=1%23attr_10496%244280%40DEFAULT",
    "핑크" : "&filter=1%23attr_10496%244288%40DEFAULT",
    "베이지" : "&filter=1%23attr_10496%244292%40DEFAULT"
}

In [22]:
# 원피스 검색
# url = 'https://www.coupang.com/np/search?component=&q=원피스'
url = 'https://www.coupang.com/np/search?rocketAll=false&searchId=928c924f13864a44aad7b67797168d82&q=%EC%9B%90%ED%94%BC%EC%8A%A4&brand=&offerCondition=&filter=1%23attr_10496%244278attr_10732%243727attr_3676%24747%40DEFAULT&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=39823&component=&rating=0&sorter=scoreDesc&listSize=36'

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36", 
          "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
res = requests.get(url, headers=headers, verify=False)

res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml") 

titles = soup.select('#productList > li')
print(len(titles))

36


In [23]:
titles[0].select_one('a > dl > dd > div > div.name').text

'셔츠원피스 여성 반소매 브이넷 블랙 베이지 하객원피스'

In [28]:
titles[0].select_one('a > dl > dt > img')

In [25]:

number = 0

for title in titles:
    number = number + 1
    product_title = title.select_one('a > dl > dt > img')['src']
    print(product_title)
#     product_title = re.sub('\n','', product_title)
#     product_title = product_title.lstrip()
#     print('[' + str(number) + ']', product_title)

//thumbnail6.coupangcdn.com/thumbnails/remote/230x230ex/image/vendor_inventory/1c37/e0d368baa819ae15feae1246d213265f98464c85b8e5215627106a58bd46.jpg
//thumbnail8.coupangcdn.com/thumbnails/remote/230x230ex/image/vendor_inventory/d2e1/5c67fb9da6afab79918560faf45fe52307413665519847ae1d51f016f356.jpg
//thumbnail9.coupangcdn.com/thumbnails/remote/230x230ex/image/rs_quotation_api/0p8otfgn/52e3ea17d13b451395260185e2bc98fb.jpg
//thumbnail7.coupangcdn.com/thumbnails/remote/230x230ex/image/vendor_inventory/ec4c/6454605356fbfead4c8922b56adf0e943638a49bd722a415b2a869578d6c.png
//thumbnail7.coupangcdn.com/thumbnails/remote/230x230ex/image/vendor_inventory/941b/5e2a1c812c7a4a4456c114326a97135083cf02c82b75a32faca154e1546e.jpg
//thumbnail6.coupangcdn.com/thumbnails/remote/230x230ex/image/vendor_inventory/3d9d/33319bc2859f4acc702ed17dd9d2fead79fec75839df057c5ec405151d8a.jpg
//thumbnail8.coupangcdn.com/thumbnails/remote/230x230ex/image/vendor_inventory/899c/5387bf9eda97a6f83fdd29e50b6a1a475a76b62f726f73

In [30]:

number = 0

for title in titles:
    number = number + 1
    product_title = title.select_one('a > dl > dd > div > div.name').text
    product_title = re.sub('\n','', product_title)
    product_title = product_title.lstrip()
    print('[' + str(number) + ']', product_title)

[1] 셔츠원피스 여성 반소매 브이넷 블랙 베이지 하객원피스
[2] 여성용 베이직 실키 슬립
[3] 피핀 여성용 믹스도트 스트링 원피스 #38212
[4] JQS 865 섹시 홀복 파티룩 클럽 하객 빅사이즈 2XL 셔링 원피스
[5] 템테이션 라운드넥 셔링 미니원피스 섹시원피스(당일배송)/1개만 구매해도 사은품 증정
[6] 필굿웨어 원피스 슬립 C5902F
[7] 버튼 자켓 원피스
[8] 요모조모 1847 시스루 브이넥 정장 원피스
[9] 러블리 끈 나시 웨이브 미니 원피스
[10] 채윤러브 여성 력셔리 롱원피스
[11] 앤더데이 부드러운 숙면 발포나염 원피스
[12] 힐링잠옷 모던 플레어 원피스 홈웨어
[13] 투디 제니 하운드 체크원피스
[14] 클럽 파티룩 긴팔 배색 미니 섹시 앞지퍼원피스
[15] 빼빼 여성용 벌룬 레이온 캉캉 롱원피스
[16] 여성용 구디프 슬림허리 브이넥 시크스타일 원피스
[17] 투디 트렌치 원피스 롱원피스 셔츠원피스 하객원피스 하객룩
[18] 페이퍼먼츠 단정한 셔츠 플레어 미디 원피스 03923
[19] 플랜데이 리드 베이직 허리 스트링 플리츠 긴팔 롱 원피스
[20] 빼빼 여성용 벨트세트 플리츠 롱원피스
[21] 프롬코 여성 깔끔한 씨링 패턴 롱 랩 원피스 FRC172
[22] 대박난박양 제시 부츠컷 데님 팬츠
[23] 티탁 빅사이즈 카라넥 단추 스트링 셔링 포켓 하프 셔츠 롱원피스
[24] 엘라운지 똥배보정 메쉬 노라인 햄팬티 3매
[25] 포플러앤씨 시크 벨트 원피스
[26] 아이엠몽 여성 타이 V넥 긴팔 니트 미니 원피스 슬림핏 데이트룩
[27] 빼빼 여성용 배색레터링 캉캉 코튼 롱 원피스
[28] 채윤러브 여름 줄무늬 스트라이프 여성 민소매 나시 롱 원피스
[29] 글램디브 볼레로 긴팔 미니원피스 보니리 AG1617
[30] 대박난박양 슬리브리스 슬림 원피스 베이지+블랙 (2개)
[31] MA 빅사이즈 하프넥 셔링 롱 원피스
[32] 캐럿 여성 프런트 셔링 롱 반팔 원피스
[33] 채윤러브 chaeyoon love 여성 

In [ ]:
product_img_url = "http:"+ soup.select_one('#repImageContainer > img')['src']
urllib.request.urlretrieve(product_img_url, "이미지.jpg")